<a href="https://colab.research.google.com/github/zeyad-mohvme-d/-Sign-Language-Fingerspelling-Assistant-ASL-Alphabet-/blob/main/efficient_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Dense, Dropout, GlobalAveragePooling2D,
    RandomRotation, RandomZoom, Rescaling, RandomContrast,
    RandomBrightness
)
from tensorflow.keras.layers import Lambda
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing import image_dataset_from_directory
import os
import numpy as np
import matplotlib.pyplot as plt
import kagglehub


path = kagglehub.dataset_download("grassknoted/asl-alphabet")
print("Dataset downloaded to:", path)


path = "/kaggle/input/asl-alphabet"


image_size = (128,128)
batch_size = 32
NUMBER_OF_CLASSES = 29

train_path = os.path.join(path, "asl_alphabet_train", "asl_alphabet_train")
test_path  = os.path.join(path, "asl_alphabet_test" , "asl_alphabet_test")


train_data = image_dataset_from_directory(
    train_path,
    validation_split=0.1,
    subset="training",
    seed=123,
    image_size=image_size,
    batch_size=batch_size
)

val_data = image_dataset_from_directory(
    train_path,
    validation_split=0.1,
    subset="validation",
    seed=123,
    image_size=image_size,
    batch_size=batch_size
)

train_data = train_data.prefetch(tf.data.AUTOTUNE)
val_data   = val_data.prefetch(tf.data.AUTOTUNE)


preprocessing_and_augmentation = Sequential([
    #Rescaling(1./255),
    RandomRotation(0.2),
    RandomZoom(0.2),
    RandomContrast(0.2),
    RandomBrightness(0.2),
    Lambda(preprocess_input),
])
inputs = Input(shape=(128, 128, 3))
x = preprocessing_and_augmentation(inputs)

base_model = EfficientNetB0(weights='imagenet', include_top=False, input_tensor=x)


for layer in base_model.layers:
    layer.trainable = False


for layer in base_model.layers[-20:]:
    layer.trainable = True


x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.4)(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(256, activation='relu')(x)
outputs = Dense(NUMBER_OF_CLASSES)(x)

model = Model(inputs, outputs)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)


history = model.fit(
    train_data,
    epochs=10,
    validation_data=val_data,
    callbacks=[early_stopping]
)
prob_model = Sequential([
    model,
    tf.keras.layers.Softmax()
])

m = {}
for i in range(26):
    m[chr(ord('A') + i)] = i

m["nothing"] = 27
m["space"]  = 28


Using Colab cache for faster access to the 'asl-alphabet' dataset.
Dataset downloaded to: /kaggle/input/asl-alphabet
Found 87000 files belonging to 29 classes.
Using 78300 files for training.
Found 87000 files belonging to 29 classes.
Using 8700 files for validation.
Epoch 1/10
2447/2447 ━━━━━━━━━━━━━━━━━━━━ 130s 48ms/step - accuracy: 0.5473 - loss: 1.6123 - val_accuracy: 0.9514 - val_loss: 0.1492
Epoch 2/10
2447/2447 ━━━━━━━━━━━━━━━━━━━━ 114s 47ms/step - accuracy: 0.9171 - loss: 0.2515 - val_accuracy: 0.9728 - val_loss: 0.0784
Epoch 3/10
2447/2447 ━━━━━━━━━━━━━━━━━━━━ 116s 47ms/step - accuracy: 0.9469 - loss: 0.1571 - val_accuracy: 0.9752 - val_loss: 0.0699
Epoch 4/10
2447/2447 ━━━━━━━━━━━━━━━━━━━━ 115s 47ms/step - accuracy: 0.9622 - loss: 0.1138 - val_accuracy: 0.9837 - val_loss: 0.0443
Epoch 5/10
2447/2447 ━━━━━━━━━━━━━━━━━━━━ 113s 46ms/step - accuracy: 0.9690 - loss: 0.0912 - val_accuracy: 0.9887 - val_loss: 0.0322
Epoch 6/10
2447/2447 ━━━━━━━━━━━━━━━━━━━━ 114s 47ms/step - accuracy

In [ ]:
model.save("EfficientV2.keras")

In [ ]:
import numpy as np

# خذ صورة واحدة من val_data
for img_batch, label_batch in val_data.take(1):
    single_image = img_batch[0:1]  # shape: (1, 128, 128, 3)
    break

# توقعات logits
logits = model.predict(single_image)
print("Logits shape:", logits.shape)  # (1, 29)
print("Logits sample:", logits[0][:5])

# توقعات احتمالات
probs = prob_model.predict(single_image)
print("Probabilities shape:", probs.shape)  # (1, 29)
print("Probabilities sample:", probs[0][:5])
print("Sum of probabilities:", np.sum(probs[0]))  # يجب أن تكون ≈ 1.0
print("Max prob:", np.max(probs[0]))
print("Predicted class:", np.argmax(probs[0]))

NameError: name 'val_data' is not defined